In [ ]:
import torch
import numpy as np
import re
import torch.nn.functional as F
import torch.nn as nn
import time
import psutil
import argparse
from CNN1 import SimpleCNN1D
# from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import torch.optim as optim
from tqdm.notebook import tqdm, trange

# 测试10折交叉验证 （分层）

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(device)

parser = argparse.ArgumentParser()
parser.add_argument("--datasets", default='BCB')
parser.add_argument("--data_setting", default='11')
parser.add_argument("--batch_size", default=32)
parser.add_argument("--epochs", default=5)
parser.add_argument("--lr", default=0.001)
parser.add_argument("--threshold", default=0.7)
parser.add_argument("--weight_decay", default=0.0001)
parser.add_argument("--input_channels", default=100)
parser.add_argument("--out_channels_1", default=64)
parser.add_argument("--out_channels_2", default=32)
parser.add_argument("--hidden_size_1", default=32)
parser.add_argument("--hidden_size_2", default=16)
parser.add_argument("--output_size", default=1)
parser.add_argument("--n_splits", default=7)
args = parser.parse_args(args=[])

mem_usage = []
gpu_usage = []

model = SimpleCNN1D(args.input_channels, args.out_channels_1, args.out_channels_2, args.hidden_size_1, args.hidden_size_2, args.output_size).to(device)

criterion = nn.BCELoss()
# criterion = nn.BCEWithLogitsLoss()
# criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)


def create_batches(data):
    # random.shuffle(data)
    batches = [data[i:i + args.batch_size] for i in range(0, len(data), args.batch_size)]
    return batches

def get_dataset(id):
    indexdir = '../input/codeclone_bcb/BCB/'
    if id == '0':
        trainfile = open(indexdir + 'traindata.txt')
        validfile = open(indexdir + 'devdata.txt')
        testfile = open(indexdir + 'testdata.txt')
    elif id == '11':
        trainfile = open(indexdir + 'traindata11.txt')
        validfile = open(indexdir + 'devdata.txt')
        testfile = open(indexdir + 'testdata.txt')
    else:
        print('file not exist')
        quit()
    trainlist = trainfile.readlines()
    print(len(trainlist))
    validlist = validfile.readlines()
    print(len(validlist))
    testlist = testfile.readlines()
    print(len(testlist))
    print("data total number:", len(trainlist)+len(validlist)+len(testlist))
    
    return trainlist, validlist, testlist


def gcj_dataset(id):
    indexdir='../input/codeclone_gcj/javadata/'
    if id=='0':
        trainfile=open(indexdir+'trainall.txt')
        validfile = open(indexdir+'valid.txt')
        testfile = open(indexdir+'test.txt')
    elif id=='13':
        trainfile = open(indexdir+'train13.txt')
        validfile = open(indexdir+'valid.txt')
        testfile = open(indexdir+'test.txt')
    elif id=='11':
        trainfile = open(indexdir+'train11.txt')
        validfile = open(indexdir+'valid.txt')
        testfile = open(indexdir+'test.txt')
    elif id=='0small':
        trainfile = open(indexdir+'trainsmall.txt')
        validfile = open(indexdir+'valid.txt')
        testfile = open(indexdir+'test.txt')
    elif id == '13small':
        trainfile = open(indexdir+'train13small.txt')
        validfile = open(indexdir+'validsmall.txt')
        testfile = open(indexdir+'testsmall.txt')
    elif id=='11small':
        trainfile = open(indexdir+'train11small.txt')
        validfile = open(indexdir+'validsmall.txt')
        testfile = open(indexdir+'testsmall.txt')
    else:
        print('file not exist')
        quit()
    trainlist=trainfile.readlines()
    print(len(trainlist))
    validlist=validfile.readlines()
    print(len(validlist))
    testlist=testfile.readlines()
    print(len(testlist))
    
    return trainlist, validlist, testlist


def train_and_valid(trainlist, y, file_path, epochs):
    epochs = trange(epochs, leave=True, desc="Epoch")
    best_f1 = None
    total_train_time = 0
#     total_valid_time = 0
    final_loss = 0
    count = 0
    for epoch in epochs:
        batches = create_batches(trainlist)
        totalloss = 0.0
        main_index = 0.0
        train_time_start = time.time()
        for index, batch in tqdm(enumerate(batches), total=len(batches), desc="Batches"):
            optimizer.zero_grad()
            batchloss = 0
            for index in batch:
                code1path = '../input/codeclone_bcb/BCB' + file_path[index][0].strip('.')
                code2path = '../input/codeclone_bcb/BCB' + file_path[index][1].strip('.')
#                 label = int(file_path[index][2])
                label = int(y[index])
                embeddings_dir = 'processed_data/embeddings_3/'  # embeddings_3是词向量序列嵌入，4是随机游走嵌入

                result_1 = re.findall(r'\d+\w', code1path)
                result_2 = re.findall(r'\d+\w', code2path)

                code1_embedding_path = embeddings_dir + result_1[0] + '.pt'
                code2_embedding_path = embeddings_dir + result_2[0] + '.pt'
                
#                 code1path = file_path[index][0]
#                 code2path = file_path[index][1] # 此处的注释为GCJ数据集
#                 label = int(y[index])
        
#                 embeddings_dir = 'processed_data/GCJ/embeddings_1/' # embeddings_1是词向量嵌入，2是随机游走嵌入
        
#                 code1_embedding_path = embeddings_dir + code1path + '.pkl.pt'
#                 code2_embedding_path = embeddings_dir + code2path + '.pkl.pt'
                
                code1_feature = torch.load(code1_embedding_path).to(device)
                code2_feature = torch.load(code2_embedding_path).to(device)
                
                if label == -1:
                    label = 0
                    
                label = torch.tensor(label, dtype=torch.float, device=device)
                label = torch.unsqueeze(label, 0)
#                 vector = torch.tensor(vector, device=device)
                
                output = model(code1_feature, code2_feature)
                output = torch.squeeze(output, 0)

                mem = psutil.virtual_memory()
                mem_usage.append(mem.used) # 记录内存的使用情况

                gpu_usage.append(torch.cuda.memory_allocated()) # 记录显存的使用情况

#                 print(output)
                batchloss = batchloss + criterion(output, label)
                
#             batchloss.backward(retain_graph=True)
            batchloss.backward()
            optimizer.step()
            loss = batchloss.item()
            totalloss += loss
            main_index = main_index + len(batch)
            loss = totalloss / main_index
            epochs.set_description("Epoch (Loss=%g)" % round(loss, 5))

        train_time_end = time.time()
        train_time = train_time_end - train_time_start
        total_train_time = total_train_time + train_time
        
        final_loss = totalloss / main_index
        
    avg_mem = sum(mem_usage) / len(mem_usage)
    avg_gpu = sum(gpu_usage) / len(gpu_usage) # 每训练一个完整周期所平均使用的显存与内存
    
    return total_train_time, final_loss, avg_mem, avg_gpu


def test(testlist, y, file_path):
    # model.eval()
    count = 0
    correct = 0
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    results = []
    for index in testlist:
        code1path = '../input/codeclone_bcb/BCB' + file_path[index][0].strip('.')
        code2path = '../input/codeclone_bcb/BCB' + file_path[index][1].strip('.')
#         label = int(file_path[index][2])
        label = int(y[index])
        embeddings_dir = 'processed_data/embeddings_3/'

        result_1 = re.findall(r'\d+\w', code1path)
        result_2 = re.findall(r'\d+\w', code2path)

        code1_embedding_path = embeddings_dir + result_1[0] + '.pt'
        code2_embedding_path = embeddings_dir + result_2[0] + '.pt'
        
#         code1path = file_path[index][0]
#         code2path = file_path[index][1] # 此处的注释为GCJ数据集
#         label = int(y[index])
        
#         embeddings_dir = 'processed_data/GCJ/embeddings_1/' # embeddings_1是词向量嵌入，2是随机游走嵌入
        
#         code1_embedding_path = embeddings_dir + code1path + '.pkl.pt'
#         code2_embedding_path = embeddings_dir + code2path + '.pkl.pt'

        code1_feature = torch.load(code1_embedding_path).to(device)
        code2_feature = torch.load(code2_embedding_path).to(device)
        
        if label == -1:
            label = 0
                    
        label = torch.tensor(label, dtype=torch.float, device=device)
        
        output = model(code1_feature, code2_feature)
        output = torch.squeeze(output, 0)
        results.append(output.item())
        prediction = output.item()

        if prediction > args.threshold and label.item() == 1:
            tp += 1
            # print('tp')
        if prediction <= args.threshold and label.item() == 0:
            tn += 1
            # print('tn')
        if prediction > args.threshold and label.item() == 0:
            fp += 1
            # print('fp')
        if prediction <= args.threshold and label.item() == 1:
            fn += 1
            # print('fn')
    print(tp, tn, fp, fn)
    p = 0.0
    r = 0.0
    f1 = 0.0
    if tp + fp == 0:
        print('precision is none')
        return
    p = tp / (tp + fp)
    if tp + fn == 0:
        print('recall is none')
        return
    r = tp / (tp + fn)
    f1 = 2 * p * r / (p + r)
    print('precision')
    print(p)
    print('recall')
    print(r)
    print('F1')
    print(f1)
    return results, p, r, f1, tp, tn, fp, fn


def main():
    
    skf = StratifiedKFold(n_splits=args.n_splits, shuffle=True, random_state=42)
    
    trainlist = []
    validlist = []
    testlist = []
    
    total_train_time = 0
    total_test_time = 0
    fold_p_scores = []
    fold_r_scores = []
    fold_f1_scores = []
    fold_avg_mem = []
    fold_avg_gpu = []
    fold_classification_statistics = {}

    if args.datasets == 'BCB':
        trainlist, validlist, testlist = get_dataset(args.data_setting) #BCB
        print("Using dataset: BCB")
    else:
        trainlist, validlist, testlist = gcj_dataset(args.data_setting) #GCJ
        print("Using dataset: GCJ")
    
    full_data = trainlist + validlist + testlist
    
    file_path = []
    labels = []
    
    for line in full_data:
        pairinfo = line.split()
        code1path = pairinfo[0].replace('\\', '/')
        code2path = pairinfo[1].replace('\\', '/')
        label=int(pairinfo[2])

#         file_path.append([code1path, code2path, label])
        file_path.append([code1path, code2path])
        labels.append(label)
    
    y = np.array(labels)
    k = 0
    train_data_len = 0
    test_data_len = 0
#     for train_index, test_index in kf.split(file_path):
    for train_index, test_index in skf.split(file_path, y):
        k += 1
        print(f'第{k}折交叉验证开始:')
        print('train data:', len(train_index))
        print('test data:', len(test_index))
        
        train_data_len = len(train_index)
        test_data_len = len(test_index)
        
        train_time, loss, avg_mem, avg_gpu = train_and_valid(train_index, y, file_path, args.epochs)
        
        test_time_start = time.time()
        testresults, test_p, test_r, test_f1, test_tp, test_tn, test_fp, test_fn = test(test_index, y, file_path)
        test_time_end = time.time()
        test_time = test_time_end - test_time_start
        
        fold_p_scores.append(test_p)
        fold_r_scores.append(test_r)
        fold_f1_scores.append(test_f1)
        fold_avg_mem.append(avg_mem)
        fold_avg_gpu.append(avg_gpu)
        fold_classification_statistics[f'{k}_fold: '] = [test_tp, test_tn, test_fp, test_fn]
        total_train_time = total_train_time + train_time
        total_test_time = total_test_time + test_time
    
    print()
    print("每折交叉验证训练数据数量：", train_data_len)
    print("每折交叉验证验证数据数量：", test_data_len)
    
    print(f"{args.n_splits}折交叉验证总训练时间:", total_train_time)
    print(f"{args.n_splits}折交叉验证总测试时间:", total_test_time)
    
    print("每折的Precision分数:", fold_p_scores)
    print("每折的Recall分数:", fold_r_scores)
    print("每折的F1分数:", fold_f1_scores)
    
    print("Precision平均分数:", np.mean(fold_p_scores))
    print("Recall平均分数:", np.mean(fold_r_scores))
    print("F1平均分数:", np.mean(fold_f1_scores))
    print("平均内存使用情况:", np.mean(fold_avg_mem))
    print("平均显存使用情况:", np.mean(fold_avg_gpu))
    
    finalfile = open('result/BCB_n_fold_cross_validation', mode='w')
#     finalfile = open('result/GCJ_n_fold_cross_validation', mode='w')
    finalfile.write('使用的数据集: ' + str(args.datasets) + '\n')
    finalfile.write('threshold: ' + str(args.threshold) + '\n')
    finalfile.write(f"{args.n_splits}_fold_train_time:" + str(total_train_time) + '\n')
    finalfile.write(f"{args.n_splits}_fold_test_time:" + str(total_test_time) + '\n')
    finalfile.write("每折交叉验证训练数据数量:" + str(train_data_len) + '\n')
    finalfile.write("每折交叉验证验证数据数量:" + str(test_data_len) + '\n')
    finalfile.write("每折交叉验证的结果:" + str(fold_classification_statistics) + '\n')
    finalfile.write('每折的Precision分数:' + str(fold_p_scores) + '\n')
    finalfile.write('每折的Recall分数:' + str(fold_r_scores) + '\n')
    finalfile.write('每折的F1分数:' + str(fold_f1_scores) + '\n')
    finalfile.write('Precision平均分数:' + str(np.mean(fold_p_scores)) + '\n')
    finalfile.write('Recall平均分数:' + str(np.mean(fold_r_scores)) + '\n')
    finalfile.write('F1平均分数:' + str(np.mean(fold_f1_scores)) + '\n')
    finalfile.write('平均内存使用情况:' + str(np.mean(fold_avg_mem) / (1024 ** 2)) + 'MB' + '\n')
    finalfile.write('平均显存使用情况:' + str(np.mean(fold_avg_gpu) / (1024 ** 2)) + 'MB' + '\n')
    finalfile.close()
    
    
main()
